In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np

# Directories for each class of images
CLASS_DIRS = ["Bacterial Blight", "Downy Mildew", "Blast", "Dead Heart", "Healthy"]

# Data directory where the images are stored
IMAGE_DIR = "dataset"

# Preprocess images by converting them to HSV and applying a mask
def preprocess_to_hsv(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    
    # Thresholding to detect disease areas in HSV
    lower_bound = np.array([25, 40, 40])  # Adjust these values based on your data
    upper_bound = np.array([85, 255, 255])
    mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
    masked_image = cv2.bitwise_and(hsv_image, hsv_image, mask=mask)
    
    return hsv_image

# Custom generator that preprocesses images to HSV
def hsv_image_generator(generator, directory, subset):
    data_gen = generator.flow_from_directory(
        directory,
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical',
        subset=subset
    )
    
    while True:
        batch_x, batch_y = next(data_gen)  # Fetch batch
        batch_x_hsv = np.array([preprocess_to_hsv(img) for img in batch_x])
        yield batch_x_hsv, batch_y

# Define ImageDataGenerator with augmentation for training
def create_train_datagen():
    return ImageDataGenerator(
        rescale=1./255,          # Normalize pixel values
        rotation_range=30,       # Randomly rotate images
        width_shift_range=0.2,   # Randomly shift images horizontally
        height_shift_range=0.2,  # Randomly shift images vertically
        shear_range=0.2,         # Apply random shear
        zoom_range=0.2,          # Apply random zoom
        horizontal_flip=True,    # Randomly flip horizontally
        vertical_flip=True,      # Randomly flip vertically
        validation_split=0.2     # 20% for validation
    )

# CNN Model Definition
def build_enhanced_cnn_model(input_shape=(256, 256, 3), num_classes=5):
    model = models.Sequential()

    # 1st Convolutional Block
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    # 2nd Convolutional Block
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    # 3rd Convolutional Block
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    # 4th Convolutional Block
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    # Flatten and Fully Connected Layers
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))  # Dropout to prevent overfitting

    # Output layer (num_classes outputs)
    model.add(layers.Dense(num_classes, activation='softmax'))

    # Compile model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Main code to train the model
if __name__ == "__main__":
    # Initialize data generators
    train_datagen = create_train_datagen()
    validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

    train_generator = hsv_image_generator(train_datagen, IMAGE_DIR, subset='training')
    validation_generator = hsv_image_generator(validation_datagen, IMAGE_DIR, subset='validation')

    # Build and summarize the CNN model
    cnn_model = build_enhanced_cnn_model(input_shape=(256, 256, 3), num_classes=len(CLASS_DIRS))
    cnn_model.summary()

    # Callbacks for early stopping and learning rate adjustment
    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    learning_rate_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-5)

    # Calculate steps per epoch
    steps_per_epoch = train_datagen.flow_from_directory(IMAGE_DIR, subset='training').samples // 32
    validation_steps = train_datagen.flow_from_directory(IMAGE_DIR, subset='validation').samples // 32

    # Train the model
    history = cnn_model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=20,  # Adjust based on needs
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        callbacks=[early_stopping, learning_rate_scheduler]
    )

    # Save the trained model
    cnn_model.save("rice_leaf_disease_classifier_hsv_refactored.h5")

    # Evaluate the model
    val_loss, val_acc = cnn_model.evaluate(validation_generator)
    print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_16 (Conv2D)                   │ (None, 254, 254, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_20               │ (None, 254, 254, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_16 (MaxPooling2D)      │ (None, 127, 127, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_17 (Conv2D)                   │ (None, 125, 125, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_21               │ (None, 125, 125, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_17 (MaxPooling2D)      │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_18 (Conv2D)                   │ (None, 60, 60, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_22               │ (None, 60, 60, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_18 (MaxPooling2D)      │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_19 (Conv2D)                   │ (None, 28, 28, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_23               │ (None, 28, 28, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_19 (MaxPooling2D)      │ (None, 14, 14, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 50176)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 256)                 │      12,845,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_24               │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 5)                   │           1,285 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,237,957 (50.50 MB)

 Trainable params: 13,236,485 (50.49 MB)

 Non-trainable params: 1,472 (5.75 KB)

Found 4723 images belonging to 5 classes.
Found 1178 images belonging to 5 classes.
Found 4723 images belonging to 5 classes.
Epoch 1/20
